In [51]:
import getpass
import os
import tiktoken

In [52]:
import sys
sys.path.append("..")  # Ajoute le dossier parent

from credentials import Azure_OpenAI_OB_Endpoint, Azure_OpenAI_OB_Key

In [53]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint= Azure_OpenAI_OB_Endpoint ,
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key= Azure_OpenAI_OB_Key,
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy",
    #streaming=True,
)

In [54]:
import tiktoken

def count_tokens(text, model="gpt-4o"):
    # Obtenir l'encodage en fonction du modèle
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        # Si le modèle n'est pas pris en charge, utilisez un encodage par défaut
        encoding = tiktoken.get_encoding("cl100k_base")
    
    # Convertir le texte en tokens
    tokens = encoding.encode(text)
    
    # Retourner le nombre de tokens
    return len(tokens)

# Exemple d'utilisation
my_text = "Voici un exemple de texte que nous voulons analyser."
token_count = count_tokens(my_text, model="gpt-4o")
print(f"Nombre de tokens : {token_count}")

Nombre de tokens : 10


## 1 - Test

In [55]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. 
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")]
)

In [56]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

contenu_ancien = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
contenu_nouveau = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
llm_output = chain.invoke(
    {
        "code": "Code monétaire et financier",
        "old_version": contenu_ancien,
        "new_version": contenu_nouveau,
        "input": "I love programming.",
    }
)

In [57]:
print(llm_output)

L'analyse comparative des deux versions de l'article révèle une modification notable. La nouvelle version de l'article a supprimé la mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" dans les deux alinéas.

**Ancienne version :**
1. Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que

## 2 - Function

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [59]:
def llm_legal_change_com_v1(old_version, new_version): 
    prompt = ChatPromptTemplate.from_messages([("system",
            """
            Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et de commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. Tu n'inventeras rien et tu te baseras uniquement sur les données fournies.
            Si tu ne sais pas ou tu ne comprends pas , dis le. 
            Inutile de reciter les textes fournis, tu ne retourneras que ton analyse.
            """),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

In [60]:
# Amélioration prompt avec o1
def llm_legal_change_com_v2(old_version, new_version): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
	Tu es un avocat et analyste juridique très expérimenté. 
    Ta mission consiste à examiner et commenter les évolutions réglementaires d’un article de loi en comparant sa version 
    antérieure avec sa version révisée.

	Consignes :
	1.	Utilise un langage juridique précis.
	2.	Base-toi uniquement sur les informations fournies, sans inventer ni extrapoler.
	3.	Si tu ne comprends pas un point ou si les informations sont insuffisantes, signale-le clairement.
	4.	Ne récite pas le texte en intégralité : concentre-toi sur l’analyse et la comparaison.

	Contenu à analyser :
	•	Ancienne version : {old_version}
	•	Nouvelle version : {new_version}

	Objectif :
    Produis un commentaire juridique concis, en mettant en évidence les principaux changements, leur portée et leurs éventuelles conséquences. N’inclus dans ta réponse que l’analyse finale.

            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

In [75]:
# Amélioration prompt avec o1
def llm_legal_change_com_v3(old_version, new_version): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
	Tu es un avocat et analyste juridique très expérimenté. 
    Ta mission consiste à examiner et commenter les évolutions réglementaires d’un article de loi en comparant sa version 
    antérieure avec sa version révisée.

	Consignes :
	1.	Utilise un langage juridique précis.
	2.	Base-toi uniquement sur les informations fournies, sans inventer ni extrapoler.
	3.	Si tu ne comprends pas un point ou si les informations sont insuffisantes, signale-le clairement.
	4.	Ne récite pas le texte en intégralité : concentre-toi sur l’analyse et la comparaison.

	Contenu à analyser :
	•	Ancienne version : {old_version}
	•	Nouvelle version : {new_version}

	Objectif :
    Produis un commentaire juridique concis, en mettant en évidence les principaux changements, leur portée et leurs éventuelles conséquences. N’inclus dans ta réponse que l’analyse finale.

            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

## 3 - Data prep

In [61]:
import pandas as pd
AML5_modifs= pd.read_excel("/Users/oussa/Desktop/Github_perso/Legal_FR_Tracker/data_output/Legifrance_DB_TrackChange.xlsx")

In [62]:
# Filter on AML text
AML5_modifs =  AML5_modifs[AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-1544")| AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-115") ]

In [ ]:
#check results
AML5_modifs['Titre Article Modificateur'].unique()

In [64]:
# Keep only changed articles in their content
AML5_modifs = AML5_modifs[AML5_modifs['Compare contenu'] =="OK change"]
AML5_modifs['Compare contenu'].unique()

array(['OK change'], dtype=object)

In [65]:
# Delete KO in Old and new articles
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Ancien_Article'].str.contains("KO", na=False)]
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Nouv_Vers_Article'].str.contains("KO", na=False)]

In [66]:
len(AML5_modifs)

58

## 4 - Apply function for each row

In [67]:
#   AML5_modifs
#   llm_legal_change_com()

In [68]:
AML5_modifs_select = AML5_modifs.iloc[:10]

### 4.1 - Application fonction 1

In [69]:
AML5_modifs_select['LLM_Change_Analysis_1'] = AML5_modifs_select.apply(
    lambda row: llm_legal_change_com_v1(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), 
    axis=1)

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_71405/819962179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_1'] = AML5_modifs_select.apply(


In [71]:
AML5_modifs_select.to_excel("/Users/oussa/Desktop/Github_perso/Legal_FR_Tracker/data_output_ipynb/output_llm_change_analysis_v1.xlsx")

### 4.2 - Application fonction 2

In [72]:
AML5_modifs_select['LLM_Change_Analysis_2'] = AML5_modifs_select.apply( 
        lambda row: llm_legal_change_com_v2(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), axis=1 ) 

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_71405/2587971769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_2'] = AML5_modifs_select.apply(


In [73]:
AML5_modifs_select.to_excel("/Users/oussa/Desktop/Github_perso/Legal_FR_Tracker/data_output_ipynb/output_llm_change_analysis_v2.xlsx")

### 4.3 - Application fonction 3

In [76]:
AML5_modifs_select['LLM_Change_Analysis_3'] = AML5_modifs_select.apply( 
        lambda row: llm_legal_change_com_v3(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), axis=1 ) 

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_71405/2334415541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_3'] = AML5_modifs_select.apply(


In [77]:
AML5_modifs_select.to_excel("/Users/oussa/Desktop/Github_perso/Legal_FR_Tracker/data_output_ipynb/output_llm_change_analysis_v3.xlsx")

## 5 - Summarize

In [78]:
all_llm_analysis = AML5_modifs_select['LLM_Change_Analysis_3'] 

In [79]:
all_llm_analysis = all_llm_analysis.to_string()

In [80]:
text_variable = AML5_modifs_select['LLM_Change_Analysis_3'].str.cat()
text_variable

'L\'analyse des versions antérieure et révisée de l\'article de loi révèle plusieurs modifications notables, tant sur le fond que sur la forme, qui méritent une attention particulière.\n\n1. **Suppression de certaines exigences spécifiques** :\n   - La nouvelle version a supprimé les exigences détaillées relatives aux dispositifs de sécurité, de contrôle interne, de gestion des conflits d\'intérêts, et de systèmes informatiques résilients et sécurisés (ancien 5°). Cette suppression pourrait indiquer une simplification des critères d\'enregistrement ou un transfert de ces exigences vers d\'autres régulations ou directives spécifiques.\n\n2. **Élimination des obligations spécifiques pour les prestataires du service mentionné au 1° de l\'article L. 54-10-2** :\n   - Les obligations spécifiques telles que la conclusion de conventions avec les clients, l\'établissement d\'une politique de conservation, la restitution des actifs numériques, la ségrégation des détentions, et l\'abstention d\'

In [81]:
token_count = count_tokens(text_variable, model="gpt-4o")
print(f"Nombre de tokens : {token_count}")

Nombre de tokens : 5805


### 5.1 - Simple summarize

In [82]:
# Amélioration prompt avec o1 + intération
def wrap_up_simple(text_to_summarize): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
		Tu es un avocat et analyste juridique très expérimenté.
		Ta mission consiste à produire un résumé global des analyses réalisées sur les évolutions réglementaires de plusieurs articles de 
  		loi en comparant leurs versions antérieures et révisées.

	Consignes :
	1.	Utilise un langage juridique précis, clair et synthétique.
	2.	Appuie-toi uniquement sur les analyses fournies pour chaque article. Ne fais aucune supposition ou extrapolation.
	3.	Regroupe et hiérarchise les informations :
	•	Identifie les thématiques ou tendances communes (par exemple : renforcement des obligations, simplification de procédures, etc.).
	•	Mets en évidence les changements les plus significatifs et leur impact global.
	4.	Si certaines analyses révèlent des ambiguïtés ou des manques d’informations, mentionne-les brièvement.

	Objectif :
 		Fournis un résumé synthétique mettant en lumière les évolutions majeures, les implications globales, et les tendances observées 
   		à travers les différents articles. Structure ta réponse de manière organisée (par exemple : par thématique ou par impact).
     
    Voici le text a résumer : {text_to_summarize}
            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "text_to_summarize": text_to_summarize})
    
    return llm_output

In [83]:
# Amélioration prompt avec o1 + intération
def wrap_up_simple(text_to_summarize): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
		Tu es un avocat et analyste juridique très expérimenté.
		Ta mission consiste à produire un résumé global des analyses réalisées sur les évolutions réglementaires de plusieurs articles de 
  		loi en comparant leurs versions antérieures et révisées.

	Consignes :
	1.	Utilise un langage juridique précis, clair et synthétique.
	2.	Appuie-toi uniquement sur les analyses fournies pour chaque article. Ne fais aucune supposition ou extrapolation.
	3.	Regroupe et hiérarchise les informations :
	•	Identifie les thématiques ou tendances communes (par exemple : renforcement des obligations, simplification de procédures, etc.).
	•	Mets en évidence les changements les plus significatifs et leur impact global.
	4.	Si certaines analyses révèlent des ambiguïtés ou des manques d’informations, mentionne-les brièvement.

	Objectif :
 		Fournis un résumé synthétique mettant en lumière les évolutions majeures, les implications globales, et les tendances observées 
   		à travers les différents articles. Structure ta réponse de manière organisée (par exemple : par thématique ou par impact).
     
    Voici le text a résumer : {text_to_summarize}
            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "text_to_summarize": text_to_summarize})
    
    return llm_output

In [84]:
summary_simple = wrap_up_simple(text_variable)
print(summary_simple)

### Résumé des Évolutions Réglementaires

#### Thématiques et Tendances Communes

1. **Simplification des Exigences et Procédures**
   - **Suppression de certaines exigences spécifiques** : Les nouvelles versions des articles de loi ont supprimé des exigences détaillées relatives aux dispositifs de sécurité, de contrôle interne, de gestion des conflits d'intérêts, et de systèmes informatiques résilients et sécurisés. Cette simplification pourrait indiquer une volonté de réduire la complexité administrative pour les prestataires de services.
   - **Simplification des exigences d'enregistrement** : La nouvelle version a supprimé l'exigence d'autorisation conformément à l'article 59 du règlement (UE) 2023/1114, ne requérant désormais que l'enregistrement préalable par l'Autorité des marchés financiers.
   - **Simplification de la procédure de suspension et de radiation** : Les détails spécifiques sur les conditions de menace grave et imminente pour la stabilité du marché des actifs numéri

### 5.2 - Custum summarize

In [85]:
# Amélioration prompt avec o1 + intération
def wrap_up_multi(text_to_summarize, audience: str, detail_level:str): 
    # audience: “professionnel” ou “tout public”
    # detail_level: “succinct” ou “détaillé”
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
		Prompt amélioré avec niveaux de détail et public cible :

		Tu es un avocat et analyste juridique très expérimenté.
		Ta mission consiste à produire un résumé global des analyses réalisées sur les évolutions réglementaires de plusieurs articles de 
  		loi en comparant leurs versions antérieures et révisées.

	Consignes générales :
	1.	Utilise un langage juridique précis et adapté au public cible défini par la variable {audience} :
	•	Si {audience} est “professionnel”, adopte un ton technique et détaillé.
	•	Si {audience} est “tout public”, simplifie les termes juridiques pour les rendre accessibles tout en conservant leur précision.
 
	2.	Ajuste la longueur et le niveau de détail du résumé en fonction de la variable {detail_level} :
	•	Si {detail_level} est “succinct”, concentre-toi uniquement sur les points clés et les impacts majeurs.
	•	Si {detail_level} est “détaillé”, développe davantage les analyses, en incluant des exemples ou des explications supplémentaires lorsque pertinent.
 
	3.	Regroupe et hiérarchise les informations :
	•	Identifie les thématiques ou tendances communes (par exemple : renforcement des obligations, simplification de procédures, etc.).
	•	Mets en évidence les changements les plus significatifs et leur impact global.
	4.	Si certaines analyses révèlent des ambiguïtés ou des manques d’informations, mentionne-les brièvement.

	Variables :
	•	Niveau de détail : {detail_level} (valeurs possibles : “succinct” ou “détaillé”)
	•	Public cible : {audience} (valeurs possibles : “professionnel” ou “tout public”)

	Objectif :
		Fournis un résumé synthétique ou détaillé, selon le niveau demandé, mettant en lumière les évolutions majeures, 
  		les implications globales, et les tendances observées à travers les différents articles. Structure ta réponse de manière organisée 
    	(par exemple : par thématique ou par impact).
    
    Voici le text  : {text_to_summarize}
            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "text_to_summarize": text_to_summarize,
        "audience": audience,
        "detail_level": detail_level})
    
    return llm_output

In [86]:
resuult_multi = wrap_up_multi(text_variable, 'professionnel', 'succinct')

In [87]:
resuult_multi

"### Résumé des Évolutions Réglementaires des Articles de Loi\n\n#### Variables :\n- **Niveau de détail** : succinct\n- **Public cible** : professionnel\n\n#### Thématiques et Tendances Communes\n\n1. **Suppression de certaines exigences spécifiques**\n   - **Ancienne version** : Exigences détaillées sur dispositifs de sécurité, contrôle interne, gestion des conflits d'intérêts, systèmes informatiques résilients et sécurisés.\n   - **Nouvelle version** : Suppression de ces exigences, potentiellement pour simplifier les critères d'enregistrement ou transférer ces obligations à d'autres régulations.\n\n2. **Élimination des obligations spécifiques pour certains prestataires**\n   - **Ancienne version** : Obligations de conclure des conventions avec les clients, établir une politique de conservation, restituer les actifs numériques, ségrégation des détentions, et abstention d'usage des actifs numériques sans consentement.\n   - **Nouvelle version** : Suppression de ces obligations, réduisa